# Projeto 2 - Classificador Automático de Sentimento

## Por: Beatriz Alexandre e Vitória de Almeida

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

###### ___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

Importando as bibliotecas

In [9]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl  
import seaborn as sns
import numpy as np
import itertools
from scipy import stats
from emoji import UNICODE_EMOJI

#### Colocando os dados em um DataFrame

Lendo o arquivo Excel

In [10]:
tw = pd.read_excel('tweets_lindt.xlsx', sheet_name = 0)

tw.head()

,Treinamento,Relevante,Positivo/Negativo
0,lindt chocolate is so overrated,1.0,0
1,@elgliko hear that pitter patter in the back g...,0.0,NaN
2,rt @statutory_boy: the guy was addicted to dru...,0.0,NaN
3,"@andreyasasylum @yourmomsuckstho hey, can you ...",1.0,1
4,great job by @noticgroup gms. come and see the...,1.0,1


#### Limpando os tweets

In [11]:
import string
ponto = string.punctuation

#lista para depoi tirar os pontos
po = ["\n", "\t"]

data_limpo = []

#dar espaço nos emojis e limpar os pontos
for tweet in tw["Treinamento"]:
    t = ''
    for palavra in tweet:
        if palavra in UNICODE_EMOJI:
            t = t+ " " + palavra + " "
        elif palavra in po:
            t += " "
        elif palavra not in ponto:
            t += palavra
    data_limpo.append(t)

tw_li =  pd.DataFrame()
tw_li['Treinamento'] = data_limpo
tw_li['Relevante'] = tw['Relevante']

tw_li.head()



,Treinamento,Relevante
0,lindt chocolate is so overrated,1.0
1,elgliko hear that pitter patter in the back gr...,0.0
2,rt statutoryboy the guy was addicted to drugs ...,0.0
3,andreyasasylum yourmomsuckstho hey can you ple...,1.0
4,great job by noticgroup gms come and see them ...,1.0


In [12]:
# tirar os @ e os #
data_limpo = []
x= ' '
for tweet in tw_li['Treinamento']:
    limp = []
    spli = tweet.split()
    for palavra in spli:
        if palavra in UNICODE_EMOJI:
            limp.append(palavra)
        elif len(palavra) > 2 and palavra[0]!='@' and palavra[0] != '#' and palavra[:4] != "http":
            limp.append(palavra)
        elif palavra == tweet[-1] and ("..." in palavra):
            limp = ''
        
    data_limpo.append(x.join(limp))
    
tw_limpo =  pd.DataFrame()
tw_limpo['Treinamento'] = data_limpo
tw_limpo['Relevante'] = tw['Relevante']
#Agumas palavras de alguns tweets travam o programa, então por isso é preciso trocar essas palavras por outra
tw_limpo['Treinamento'] = tw_limpo['Treinamento'].str.replace('gorgeous\xa0i', 'banana')


tw_limpo.head()

,Treinamento,Relevante
0,lindt chocolate overrated,1.0
1,elgliko hear that pitter patter the back groun...,0.0
2,statutoryboy the guy was addicted drugs his li...,0.0
3,andreyasasylum yourmomsuckstho hey can you ple...,1.0
4,great job noticgroup gms come and see them and...,1.0


Lista de todas as palavras dos tweets

In [13]:
pala = []

for i in tw_limpo['Treinamento']:
    frase = i.split()
    for p in frase:
        if p not in pala:
            pala.append(p)


Contar quantas vezes aparace o 1 (Relevante) e o 0 (não relevante):

In [14]:
#contador
Rel = 0
Irre = 0 

for l in range(len(tw_limpo)):
    linha = tw_limpo["Treinamento"][l].split(" ")
    for i in linha:

        if tw_limpo["Relevante"][l] == 1:
            Rel += 1
        else:
            Irre += 1

Dicionários para calcular a frequencia de relevantes e irrelevantes:

In [15]:
#adicionou as palavras no dicionário de frequencia
r_freq = {}
ir_freq = {}

for palavras in pala:
    r_freq[palavras] = 1
    ir_freq[palavras] = 1

for g in range(len(tw_limpo)):
    linha = tw_limpo["Treinamento"][g].split(" ")
    for v in linha:
        if tw_limpo["Relevante"][g] == 0:
            ir_freq[v]+=1
        else:
            r_freq[v]+= 1


A probabilidade de ser relevante da cada palavra

In [16]:
r_prob = {}
ir_prob = {}

#pala é a lista de palavras dos tweets

for palavra in pala:
    r_prob[palavra] = (r_freq[palavra]+1)/(len(pala)+Rel)
    ir_prob[palavra] = (ir_freq[palavra]+1)/(len(pala)+Irre)
    
print(r_prob['lindt'],r_freq[palavra]+1,len(pala),Rel)
print(ir_prob['lindt'],ir_freq[palavra]+1,len(pala),Irre)

0.030652280529671408 2 1906 2172
0.0050335570469798654 3 1906 2266


P(relevante) e P(irrelevante)

In [17]:
irrelevante = 0
relevante = 0 
for m in tw_limpo['Relevante']:
    if m == 0:
        irrelevante += 1
    else:
        relevante += 1 
Pr = relevante/len(tw_limpo['Relevante'])
Pi = 1-Pr



___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

# TESTE

In [18]:
# g é o DataFrame do Teste!
g = pd.read_excel('tweets_lindt.xlsx', sheet_name = 1)
g.head()

,Teste,Relevante,Positivo/Negativo
0,rt @thembarumbu: dear ladies\nif you catch ur ...,0,NaN
1,rt @dailytelegraph: one of the most notorious ...,1,0/1
2,commercial law's bitch ...is what i am right n...,0,NaN
3,@hloni_maniers i'm craving lindt lindor 😫,1,1
4,rt @lutho__m: if you're reading this tweet htt...,0,NaN


Limpando os tweets do teste

In [19]:
g.Teste = g.Teste.str.replace('@', '')
g.Teste = g.Teste.str.replace(':', '')
g.Teste = g.Teste.str.replace('-', '')
g.Teste = g.Teste.str.replace('_', '')
g.Teste = g.Teste.str.replace(',', '')
g.Teste = g.Teste.str.replace('!', '')
g.Teste = g.Teste.str.replace('(', '')
g.Teste = g.Teste.str.replace(')', '')
g.Teste = g.Teste.str.replace('=', '')
g.Teste = g.Teste.str.replace('*', '')
g.Teste = g.Teste.str.replace('#', '')

g.head()

,Teste,Relevante,Positivo/Negativo
0,rt thembarumbu dear ladies\nif you catch ur bo...,0,NaN
1,rt dailytelegraph one of the most notorious ba...,1,0/1
2,commercial law's bitch ...is what i am right n...,0,NaN
3,hlonimaniers i'm craving lindt lindor 😫,1,1
4,rt luthom if you're reading this tweet https//...,0,NaN


In [20]:
g_espaco_emoji = []
data = g["Teste"]
for tweet in data:
    t = ''
    for palavra in tweet:
        if palavra in UNICODE_EMOJI:
            t = t+ " " + palavra + " "
        else:
            t += palavra
    g_espaco_emoji.append(t)

g_limpo =  pd.DataFrame()
g_limpo['Teste'] = g_espaco_emoji
g_limpo['Relevante'] = g['Relevante']

g_limpo.head()

,Teste,Relevante
0,rt thembarumbu dear ladies\nif you catch ur bo...,0
1,rt dailytelegraph one of the most notorious ba...,1
2,commercial law's bitch ...is what i am right n...,0
3,hlonimaniers i'm craving lindt lindor 😫,1
4,rt luthom if you're reading this tweet https//...,0


Calculando as Probabilidades no Teste

In [21]:
lista_teste = []
j = []
k = []
for f in g_limpo['Teste']:
    comR = 1
    comIr = 1
    x = f.split(' ')
    
    for palavra in x:
        
        if palavra in ir_prob:
            comIr *= ir_prob[palavra]
        else:
            comIr *= 1/(len(pala)+Irre)
        if palavra in r_prob:
            comR *= r_prob[palavra]
        else:
            comR *= 1/(len(pala)+Rel)

    Rp = comR*Pr
    Irp = comIr*Pi
    
    j.append(Rp)
    k.append(Irp)
    
    if Rp >= Irp:
        lista_teste.append(1)
    else:
        lista_teste.append(0)
min(k)
g["lista teste"] = lista_teste


Imprimir as probabilidades de positivo e negativo verdadeiros e falsos

In [22]:
uu = g.loc[(g['Relevante'] == 1 ) & (g['lista teste'] == 1 ), ['Relevante', 'lista teste']]
uz = g.loc[(g['Relevante'] == 0 ) & (g['lista teste'] == 1 ), ['Relevante', 'lista teste']]
zz = g.loc[(g['Relevante'] == 0 ) & (g['lista teste'] == 0 ), ['Relevante', 'lista teste']]
zu = g.loc[(g['Relevante'] == 1 ) & (g['lista teste'] == 0 ), ['Relevante', 'lista teste']]

posV = (len(uu)/len(g['Relevante']))*100
posF = (len(uz)/len(g['Relevante']))*100
negV = (len(zz)/len(g['Relevante']))*100
negF = (len(zu)/len(g['Relevante']))*100

print('Positivos verdadeiros = {:.3f}%'.format(posV))
print('Positivos falsos = {:.3f}%'.format(posF))
print('Negativos verdadeiros = {:.3f}%'.format(negV))
print('Negativos falsos = {:.3f}%'.format(negF))

Positivos verdadeiros = 44.000%
Positivos falsos = 20.000%
Negativos verdadeiros = 33.000%
Negativos falsos = 3.000%


___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


# Conclusão

  Depois de ter o código classificador pronto e utilizá-lo para o Teste, concluise-se que a marca Lindt tem uma maior probabilidade de seus tweets serem negativos, por um lado é ruim para marca, pois quer dizer que seus clientes não estão satisfeitos com os produtos. Porém alguns tweets positivos são contados como negativos no classificador, mas isso não é necessariamente prejudicial, pois no momento em que a empresa for analisar os tweets ela apenas o elimina e garante que todos os negativos serão analisados.
  
  A Lindt deveria continuar a financiar o projeto, pois com ele, eles conseguem saber o que o seu público alvo pensa sobre seus produtos. Logo, a Lindt sabe o que deve aprimorar e o que continuar a fazer. Para ele ser ainda mais eficaz é bom criar outras categorias além de relevante e irrelevante.
  
  Não se pode alimentar a base de treinamento usando o próprio classificador, pois o código classificador foi criado com base no Treinamento e por mais que aplique se novos tweets a base é a mesma

# Referências

* Split: https://www.tutorialspoint.com/python/string_split.htm
* Tutorial
* Naive-Bayes: https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/